In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain import hub as prompts
from langsmith.client import Client
from langchain.agents import (
    AgentExecutor,
    create_react_agent,
)
from langchain_core.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

d:\LEARN\LangChain Basis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cd ..

d:\LEARN\LangChain Basis


d:\LEARN\LangChain Basis\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime  # Import datetime module to get current time

    now = datetime.datetime.now()  # Get current time
    return now.strftime("%I:%M %p")  # Format time in H:MM AM/PM format

In [5]:
# List of tools available to the agent
tools = [
    Tool(
        name="Time",  # Name of the tool
        func=get_current_time,  # Function that the tool will execute
        # Description of the tool
        description="Useful for when you need to know the current time",
    ),
]

In [14]:
# Pull the prompt template from the hub
# ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react

# client = Client()
# prompt = client.get_prompt("hwchase17/react")
# prompt

Prompt(repo_handle='react', description='', readme=None, id='228570f9-c548-44a8-bc5f-9ee8f7c2e980', tenant_id='97591f89-2916-48d3-804e-20cab23f91aa', created_at=datetime.datetime(2023, 9, 20, 23, 30, 56, 918094), updated_at=datetime.datetime(2023, 9, 20, 23, 30, 58, 201686), is_public=True, is_archived=False, tags=['StringPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner='hwchase17', full_name='hwchase17/react', num_likes=64, num_downloads=1858695, num_views=73302, liked_by_auth_user=False, last_commit_hash='d15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None)

In [6]:
prompt = prompts.pull("hwchase17/react")
prompt

d:\LEARN\LangChain Basis\.venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [7]:
# Initialize a LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [8]:
# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

In [9]:
# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [10]:
# Run the agent with a test query
response = agent_executor.invoke({"input": "What time is it?"})



> Entering new AgentExecutor chain...
Thought: I need to know the current time.
Action: Time
Action Input: 01:31 PMThought: I now know the final answer
Final Answer: It is 01:31 PM. 


> Finished chain.


In [11]:
# Print the response from the agent
print("response:", response)

response: {'input': 'What time is it?', 'output': 'It is 01:31 PM.'}
